In [ ]:
!pip install openai diffusers gtts gradio
!apt install imagemagick #Install imagemagick
!pip install moviepy
!rm /etc/ImageMagick-6/policy.xml
from openai import OpenAI
#from api_key import API_KEY
import gradio as gr
import torch
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, EulerDiscreteScheduler
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file

import os
import numpy as np
import re
from requests import get
import urllib.request
from gtts import gTTS
from moviepy.editor import *
from moviepy import config
#from api_key import API_KEY
config.IMAGEMAGICK_BINARY = '/usr/bin/convert' #Set imagemagick path


API_KEY = "YOUR OPENAI KEY"
client = OpenAI(api_key = API_KEY)

def get_script(topic):
    prompt = ("Create youtube video script about  "+topic+" separated by enters without text before or after",    )


    response = client.chat.completions.create(model="gpt-4o-mini",
        messages=[{'role':"system", "content":"you are a social media influencer excel in creating quality video scripts"},
                {'role':'user','content':[{'type':'text','text':prompt},]}],

        max_tokens=8000,
        temperature=1.0,
        top_p=1.0,
        n=1,
    )
    print(response.choices[0].message.content)
    return response.choices[0].message.content


base = "stabilityai/stable-diffusion-xl-base-1.0"
repo = "ByteDance/SDXL-Lightning"
ckpt = "sdxl_lightning_4step_unet.safetensors" # Use the correct ckpt for your step setting!

# Load model.
unet = UNet2DConditionModel.from_config(base, subfolder="unet").to("cuda", torch.float16)
unet.load_state_dict(load_file(hf_hub_download(repo, ckpt), device="cuda"))
pipe = StableDiffusionXLPipeline.from_pretrained(base, unet=unet, torch_dtype=torch.float16, variant="fp16").to("cuda")

# Ensure sampler uses "trailing" timesteps.
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing")

def gradio_app(main_key ):
  progress = gr.Progress()
  progress(0, desc="Starting...")
  text = get_script(main_key)
  #text = """ Hey there, history buffs! Did you know that Ancient Greece was more than just togas and temples? Here are three mind-blowing facts about the land of the gods! Fact number one: The Parthenon wasn't just a temple—it was also a giant treasury! 💰 This iconic structure held the wealth of Athens, making it the city's most secure bank vault. Fact number two: Ancient Greeks loved theater! 🎭 They invented drama, and their open-air theaters could hold up to 15,000 people. That's like half a stadium packed with fans of tragedies and comedies! Fact number three: The Olympics started in Ancient Greece around 776 BC. 🏅 But get this—back then, the athletes competed completely...naked! Yep, it was all about showing off their strength and skill."""
  # Split the text by , and .
  paragraphs = re.split(r"[,.?:]", text)

  #Create Necessary Folders
  if not os.path.exists("audio"):
    os.makedirs("audio")
    os.makedirs("images")
    os.makedirs("videos")

  # Loop through each paragraph and generate an image for each
  i=1
  for id in progress.tqdm(range(len(paragraphs))):
      progress((id/len(paragraphs)),desc="Generating data")
      para = paragraphs[id]
      if para == ''or para == ' ':
        continue
      pipe(main_key, height=720, width=1280, num_inference_steps=4, guidance_scale=0).images[0].save("/content/images/image"+str(i)+".png")
      # Create gTTS instance and save to a file
      tts = gTTS(text=para, lang='en', slow=False)
      tts.save(f"/content/audio/voiceover{i}.mp3")
      print("The Paragraph Converted into VoiceOver & Saved in Audio Folder!")

      # Load the audio file using moviepy
      print("Extract voiceover and get duration...")
      audio_clip = AudioFileClip(f"/content/audio/voiceover{i}.mp3")
      audio_duration = audio_clip.duration

      # Load the image file using moviepy
      print("Extract Image Clip and Set Duration...")
      image_clip = ImageClip(f"/content/images/image{i}.png").set_duration(audio_duration)

      # Use moviepy to create a text clip from the text
      print("Customize The Text Clip...")
      words = re.split(r"[ ]", para)
      word_counter = 0
      for word in words:
        if word == '' or word == ' ':
          continue
        word_counter +=1
        if word_counter == 7:
          para = para.replace(word, f"\n{word}")
          word_counter = 0
      text_clip = TextClip(para, fontsize=30, color="white")
      #text_clip = text_clip.set_pos('bottom').set_duration(audio_duration)
      # Create a black background with the same size as the text clip
      bg_clip = text_clip.on_color(size=(text_clip.w + 20, text_clip.h + 10), color=(0, 0, 0), col_opacity=1)

      # Set the position and duration
      text_with_bg = CompositeVideoClip([bg_clip, text_clip.set_pos(('center', 'center'))])
      text_with_bg = text_with_bg.set_pos('bottom').set_duration(audio_duration)

      # Use moviepy to create a final video by concatenating
      # the audio, image, and text clips
      print("Concatenate Audio, Image, Text to Create Final Clip...")
      clip = image_clip.set_audio(audio_clip)
      video = CompositeVideoClip([clip, text_with_bg])


      # Save the final video to a file
      video = video.write_videofile(f"/content/videos/video{i}.mp4", fps=24)
      print(f"The Video{i} Has Been Created Successfully!")
      i+=1


  clips = []
  l_files = os.listdir("videos")
  for file in l_files:
      clip = VideoFileClip(f"videos/{file}")
      clips.append(clip)
  progress(0.99,desc="Concatenating Parts")
  final_video = concatenate_videoclips(clips, method="compose")
  final_video = final_video.write_videofile("final_video.mp4")
  progress(1,desc="Done")
  return "/content/final_video.mp4"

# Gradio interface
iface = gr.Interface(
    fn=gradio_app,
    inputs="text",
    outputs="video",
    title="Text to Video Generator",
    description="Enter some text and get a video!",
    examples=[["Cats"],["Accient Rome"],["Free Palastine"]],
)

iface.launch(debug=True)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
rm: cannot remove '/etc/ImageMagick-6/policy.xml': No such file or directory


  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c1040d4011ac3c5bb8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


  0%|          | 0/4 [00:00<?, ?it/s]

The Paragraph Converted into VoiceOver & Saved in Audio Folder!
Extract voiceover and get duration...
Extract Image Clip and Set Duration...
Customize The Text Clip...
Concatenate Audio, Image, Text to Create Final Clip...
Moviepy - Building video /content/videos/video1.mp4.
MoviePy - Writing audio in video1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/videos/video1.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/video1.mp4
The Video1 Has Been Created Successfully!


...

MoviePy - Done.
Moviepy - Writing video /content/videos/video15.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/video15.mp4
The Video15 Has Been Created Successfully!
Moviepy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_video.mp4



t:  10%|█         | 145/1421 [00:09<00:36, 34.67it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file videos/video2.mp4, 2764800 bytes wanted but 0 bytes read,at frame 147/148, at time 6.12/6.14 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  26%|██▌       | 370/1421 [00:15<00:30, 33.92it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file videos/video4.mp4, 2764800 bytes wanted but 0 bytes read,at frame 223/224, at time 9.29/9.33 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  32%|███▏      | 450/1421 [00:19<00:47, 20.58it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file videos/video12.mp4, 2764800 bytes wanted

Moviepy - Done !
Moviepy - video ready final_video.mp4
